### 문제 7-1 : LangGraph ReAct Agent 실습 연습문제 (Vector DB + Tool 연동)

In [ ]:
import os
import warnings
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import Image, display
from typing import List

# 환경 설정
load_dotenv()
warnings.filterwarnings("ignore")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


# Vector DB 초기화 (카페 메뉴 DB)
embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
cafe_db = FAISS.load_local(
    "./db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)


In [ ]:

# 1단계: 카페 메뉴 검색 도구 정의
@tool
def search_cafe_menu(query: str) -> str:
    """카페 메뉴에서 정보를 검색합니다."""
    docs = cafe_db.similarity_search(query, k=6)
    
    formatted_docs = "\n\n---\n\n".join([
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in docs
    ])
    
    if len(docs) > 0:
        return formatted_docs
    return "관련 카페 메뉴 정보를 찾을 수 없습니다."


In [ ]:

# 2단계: 상태 정의
class AgentState(MessagesState):
    """ReAct Agent의 상태를 정의하는 클래스"""
    pass

# 3단계: LLM과 도구 설정
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

tools = [search_cafe_menu]
llm_with_tools = llm.bind_tools(tools=tools)

# 4단계: 시스템 프롬프트 정의
system_prompt = """
당신은 지금부터, 고객의 메뉴 관련 질문에 정확하게, 친절하게 답변하는 Cafe 어시스턴트입니다.

다음 도구를 사용할 수 있습니다:
- search_cafe_menu: 카페 메뉴 정보를 검색할 때 사용

고객의 질문을 파악하여 요구사항을 이해한 후, 필요한 경우 도구를 사용하여 data를 찾아 적절한 답변을 제공해 주시길 바랍니다.
"""

# 5단계: 노드 함수들 정의
def call_model(state: AgentState):
    """LLM을 호출하여 응답을 생성하는 노드"""
    print("--- Agent 노드: LLM 호출 ---")
    
    # 시스템 메시지 추가
    messages = [SystemMessage(content=system_prompt)] + state['messages']
    response = llm_with_tools.invoke(messages)
    
    print(f"LLM 응답 생성 완료. 도구 호출 여부: {bool(response.tool_calls)}")
    
    return {"messages": [response]}

In [ ]:

# 6단계: 그래프 구성
builder = StateGraph(AgentState)

# 노드 및 엣지 추가
builder.add_node("agent", call_model)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "agent")
builder.add_conditional_edges(
    "agent",
    tools_condition,  # 자동으로 도구 호출 여부 판단
)
builder.add_edge("tools", "agent")

# 7단계: 그래프 표시
custom_react_agent = builder.compile()
try:
    display(Image(custom_react_agent.get_graph().draw_mermaid_png()))
except:
    print("그래프 시각화를 건너뜁니다.")

In [ ]:

# 8단계: Agent 테스트 함수
def test_custom_react_agent():
    """사용자 정의 ReAct Agent 테스트"""
    test_questions = [
        "콜드브루와 핫브루의 차이점과 가격은 어떻게 되나요?",
        "카페 라떼와 카푸치노 중 어떤 음료가 더 부드럽고 고소한 맛인가요?",
        "에스프레소와 리스트레또의 주요 차이점은 무엇인가요?",
        "디카페인 커피와 일반 커피의 맛과 향 차이점을 알려주세요.",
        "오늘의 스페셜 음료 메뉴와 일반 음료의 특징을 비교해 주세요."
    ]
    
    for question in test_questions:
        print(f"\n{'='*60}")
        print(f"질문: {question}")
        print('='*60)
        
        # Agent 실행
        inputs = {"messages": [HumanMessage(content=question)]}
        result = custom_react_agent.invoke(inputs)
        
        # 실행 과정 출력
        print("\n=== 실행 과정 ===")
        for i, message in enumerate(result['messages']):
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"{i+1}. {type(message).__name__}: 도구 호출 - {message.tool_calls[0]['name']}")
            else:
                content_preview = message.content[:100] + "..." if len(message.content) > 100 else message.content
                print(f"{i+1}. {type(message).__name__}: {content_preview}")
        
        # 최종 답변 출력
        print(f"\n=== 최종 답변 ===")
        print(result['messages'][-1].content)

In [ ]:
# 9단계: 테스트 실행
print("\n=== 테스트 실행 ===")
test_custom_react_agent()